In [21]:
import cv2
import numpy as np

In [27]:
# Установка начальных значений для координат x и y и k
x, y, k = 200, 200, -1

# Инициализация видеокамеры
cap = cv2.VideoCapture(0)

# Функция для обработки входных данных от мыши
def take_inp(event, x1, y1, flag, param):
    global x, y, k
    if event == cv2.EVENT_LBUTTONDOWN:
        # Запоминаем координаты x и y при щелчке мыши
        x = x1
        y = y1
        # Устанавливаем k = 1, чтобы выйти из цикла while
        k = 1

# Создание окна с именем "enter_point" и установка функции take_inp в качестве обработчика событий мыши
cv2.namedWindow("enter_point")
cv2.setMouseCallback("enter_point", take_inp)

# Цикл для получения начальной точки объекта
while True:
    # Получение текущего кадра видеопотока
    _, inp_img = cap.read()
    # Отражение изображения по горизонтали
    inp_img = cv2.flip(inp_img, 1)
    # Преобразование изображения в оттенки серого
    gray_inp_img = cv2.cvtColor(inp_img, cv2.COLOR_BGR2GRAY)
    
    # Отображение текущего кадра и ожидание выбора начальной точки мышью
    cv2.imshow("enter_point", inp_img)
    
    if k == 1 or cv2.waitKey(30) == 27:
        # Если k = 1 или пользователь нажал клавишу Esc, закрываем окно и выходим из цикла
        cv2.destroyAllWindows()
        break

# Инициализация переменных для оптического потока
stp = 0
old_pts = np.array([[x, y]], dtype=np.float32).reshape(-1,1,2)
mask = np.zeros_like(inp_img)

# Цикл для отслеживания движения объекта
while True:
    # Получение текущего кадра видеопотока
    _, new_inp_img = cap.read()
    # Отражение изображения по горизонтали
    new_inp_img = cv2.flip(new_inp_img, 1)
    # Преобразование изображения в оттенки серого
    new_gray = cv2.cvtColor(new_inp_img, cv2.COLOR_BGR2GRAY)
    
    # Расчет оптического потока между двумя последовательными кадрами
    new_pts, status, err = cv2.calcOpticalFlowPyrLK(gray_inp_img, new_gray, old_pts, None, maxLevel=1,
                                                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                                               15, 0.08))

    # Обработка каждой пары старых и новых точек, возвращаемых функцией calcOpticalFlowPyrLK
    for i, j in zip(old_pts, new_pts):
        x,y = j.ravel() # Разделение координат новой точки
        a,b = i.ravel() # Разделение координат старой точки
        
        if cv2.waitKey(2) & 0xff == ord('q'): 
            stp = 1 # Если нажата клавиша 'q', остановить рисование линии
            
        elif cv2.waitKey(2) & 0xff == ord('w'):
            stp = 0  # Если нажата клавиша 'w', продолжить рисование линии
        
        elif cv2.waitKey(2) == ord('n'): 
            mask = np.zeros_like(new_inp_img) # Если нажата клавиша 'n', очистить маску
            
        if stp == 0:
            mask = cv2.line(mask, (int(a),int(b)), (int(x),int(y)), (0,0,255), 6)

        cv2.circle(new_inp_img, (int(x),int(y)), 6, (0,255,0), -1)
    # Объединяем маску и новый кадр с помощью addWeighted для создания эффекта "следа"
    new_inp_img = cv2.addWeighted(mask, 0.3, new_inp_img, 0.7, 0)
    # Отображаем инструкцию по управлению, новый кадр и маску
    cv2.putText(mask, "'q' to gap 'w' - start 'n' - clear", (10,50), 
                cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
    cv2.imshow("ouput", new_inp_img)
    cv2.imshow("result", mask)
    
    # Обновляем старые значения координат и изображений
    gray_inp_img = new_gray.copy()
    old_pts = new_pts.reshape(-1,1,2)
    # Если нажата клавиша Esc, завершаем цикл и закрываем окна
    if cv2.waitKey(1) & 0xff == 27:
        break

cv2.destroyAllWindows()
cap.release()
